In [1]:
%pip install transformers 

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Admin\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
%pip install coca-pytorch
%pip install vit-pytorch>=0.40.2
%pip install einops

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Admin\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Admin\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Admin\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [ ]:
#OmniVec (VIT) top 1
#coca-pytorch

In [1]:
import torch

# import vision transformer

from vit_pytorch.simple_vit_with_patch_dropout import SimpleViT
from vit_pytorch.extractor import Extractor

vit = SimpleViT(
    image_size = 256,
    patch_size = 32,
    num_classes = 1000,
    dim = 1024,
    depth = 6,
    heads = 16,
    mlp_dim = 2048,
    patch_dropout = 0.5  # https://arxiv.org/abs/2212.00794
)

vit = Extractor(vit, return_embeddings_only = True, detach = False)

# extractor will enable it so the vision transformer returns its embeddings

# import CoCa and instantiate it

from coca_pytorch.coca_pytorch import CoCa

coca = CoCa(
    dim = 512,                     # model dimension
    img_encoder = vit,             # vision transformer - image encoder, returning image embeddings as (batch, seq, dim)
    image_dim = 1024,              # image embedding dimension, if not the same as model dimensions
    num_tokens = 20000,            # number of text tokens
    unimodal_depth = 6,            # depth of the unimodal transformer
    multimodal_depth = 6,          # depth of the multimodal transformer
    dim_head = 64,                 # dimension per attention head
    heads = 8,                     # number of attention heads
    caption_loss_weight = 1.,      # weight on the autoregressive caption loss
    contrastive_loss_weight = 1.,  # weight on the contrastive loss between image and text CLS embeddings
).cuda()

# mock text and images

text = torch.randint(0, 20000, (4, 512)).cuda()
images = torch.randn(4, 3, 256, 256).cuda()

# train by giving CoCa your text and images with `return_loss = True`

loss = coca(
    text = text,
    images = images,
    return_loss = True  # set this to True to get the full caption + contrastive loss
)

loss.backward()

# do the above for as much text and images...
# then you can get the caption logits as so

logits = coca(
    text = text,
    images = images
) # (4, 512, 20000)

# and the CLIP-like text and image embeddings as

text_embeds, image_embeds = coca(
    text = text,
    images = images,
    return_embeddings = True
) # (4, 512), (4, 512)

ModuleNotFoundError: No module named 'torch'

In [9]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from transformers import ViTModel, ViTFeatureExtractor
from PIL import Image
import json

# Load and preprocess the image using ViT feature extractor
def preprocess_image(image_path, feature_extractor):
    image = Image.open(image_path)
    # Resize and normalize the image for ViT
    inputs = feature_extractor(images=image, return_tensors="pt")
    return inputs

def normalize_bbox(points, image_width, image_height):
    # Extract (x_min, y_min), (x_max, y_max)
    x_min, y_min = points[0]
    x_max, y_max = points[1]
    
    # Normalize the bounding box coordinates
    x_min /= image_width
    x_max /= image_width
    y_min /= image_height
    y_max /= image_height
    
    return [x_min, y_min, x_max, y_max]
# Convert bounding box to a suitable format for the model
def encode_bbox(bbox, label, num_classes=10):
    bbox = torch.tensor(bbox)  # Assume normalized bbox
    # One-hot encode the class label
    class_embedding = torch.nn.functional.one_hot(torch.tensor(label), num_classes=num_classes).float()
    bbox_token = torch.cat((bbox, class_embedding))
    return bbox_token

# Define the model architecture
class BBoxVerificationModelViT(nn.Module):
    def __init__(self, num_classes=2):
        super(BBoxVerificationModelViT, self).__init__()
        # Load pre-trained ViT backbone
        self.vit = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
        
        # Bounding box encoder (4 bbox coordinates + class label)
        self.fc_bbox = nn.Linear(4 + num_classes, 64)  # Bbox + class label token
        
        # Fully connected layers to combine ViT features and bbox token
        self.fc_combined = nn.Linear(768 + 64, 256)
        self.fc_out = nn.Linear(256, num_classes)  # Binary classification head

    def forward(self, image_patches, bbox_token):
        # Pass image patches through ViT model
        vit_features = self.vit(**image_patches).last_hidden_state[:, 0, :]  # CLS token from ViT
        
        # Encode the bounding box
        bbox_embedding = self.fc_bbox(bbox_token)
        
        # Combine ViT features with bbox embedding
        combined_features = torch.cat((vit_features, bbox_embedding), dim=1)
        
        # Classification head to predict if bbox is correct
        x = torch.relu(self.fc_combined(combined_features))
        output = self.fc_out(x)
        
        return output

# Example data loading function from Label Studio JSON format
def load_labelme_data(json_file, feature_extractor):
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    image_path = directory+data['imagePath']
    annotations = data['shapes']
    image_width = data['imageWidth']
    image_height = data['imageHeight']
    
    # Preprocess image for ViT
    image_patches = preprocess_image(image_path, feature_extractor)
    
    # Prepare bbox tokens (assume labels are numeric or convert them to numeric)
    bbox_tokens = []
    label_map = {"cat": 0, "dog": 1}  # Example label mapping
    
    for ann in annotations:
        bbox = normalize_bbox(ann['points'], image_width, image_height)
        label = label_map[ann['label']]  # Convert label to numeric
        bbox_token = encode_bbox(bbox, label)
        bbox_tokens.append(bbox_token)
    
    # Stack bbox tokens into a tensor
    bbox_tokens = torch.stack(bbox_tokens)
    
    return image_patches, bbox_tokens


directory = 'train_data/'

# Initialize model, feature extractor, and sample inputs
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
model = BBoxVerificationModelViT(num_classes=2)  # Binary classification: Correct/Incorrect

# Example LabelMe format JSON file
json_file = directory+'kak-proverit-pasport-3desk.qln0yi.json'

# Load data
image_patches, bbox_tokens = load_labelme_data(json_file, feature_extractor)

# Make predictions
output = model(image_patches, bbox_tokens)
prediction = torch.softmax(output, dim=1)
print("Prediction: ", prediction)

d:\PycharmProjects\CVTransformers\.venv\Lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


AttributeError: 'PngImageFile' object has no attribute 'T'

In [6]:
!pwd

"pwd" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.
